# Data Preparation

---

This notebook outlines the process for preparing the Telco customer data for exploratory analysis. This notebook also demonstrates the steps to create and test the functions in the util/prepare.py file.

---

## Acquire the Data

Let's first use the functions we created in acquire.ipynb to acquire our data.

In [1]:
import os
import pandas as pd
from env import username, password, hostname
database_name = 'telco_churn'

def get_db_url(database_name, username = username, password = password, hostname = hostname):
    return f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}'

def get_telco_sql():
    return '''
        SELECT *
        FROM customers
        JOIN payment_types USING (payment_type_id)
        JOIN internet_service_types USING (internet_service_type_id)
        JOIN contract_types USING (contract_type_id);
    '''

def get_telco_data(use_cache = True):
    # If the file is cached, read from the .csv file
    if os.path.exists('telco.csv') and use_cache:
        print('Using cache')
        return pd.read_csv('telco.csv')
    
    # Otherwise read from the mysql database
    else:
        print('Reading from database')
        df = pd.read_sql(get_telco_sql(), get_db_url('telco_churn'))
        df.to_csv('telco.csv', index = False)
        return df

In [2]:
telco_customers = get_telco_data()
telco_customers.head(2)

Using cache


,contract_type_id,internet_service_type_id,payment_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,payment_type,internet_service_type,contract_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,Mailed check,DSL,One year
1,1,1,2,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Mailed check,DSL,Month-to-month


---

## Identify Missing Values

We will first check if any missing values exist in our telco_customers dataframe.

In [3]:
telco_customers.isna().sum()

contract_type_id            0
internet_service_type_id    0
payment_type_id             0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
payment_type                0
internet_service_type       0
contract_type               0
dtype: int64

We do not have null values so we can proceed.

---

## Remove Duplicates

We will run drop_duplicates just in case we have duplicate rows in our data.

In [4]:
telco_customers.shape

(7043, 24)

In [5]:
telco_customers = telco_customers.drop_duplicates()
telco_customers.shape

(7043, 24)

It appears we did not have any duplicated rows.

---

## Identify Useless/Redundant Features

Next we want to identify any columns that will undoubtedly not be useful to us. This can include unique identifiers since this is no different than an index or duplicate columns such as foreign key columns. We'll simply look at our columns and make a judgement call to determine which columns can be removed.

In [6]:
telco_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   contract_type_id          7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   payment_type_id           7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

The columns contract_type_id and contract_type might be the same. Let's look at the values in these columns.

In [7]:
columns = ['contract_type_id', 'contract_type']
telco_customers[columns].value_counts()

contract_type_id  contract_type 
1                 Month-to-month    3875
3                 Two year          1695
2                 One year          1473
dtype: int64

This indeed looks like contract_type_id was a foreign key. We will keep contract_type since this is a more readable column.

Next let's look at internet_service_type_id and internet_service_type since this may be a similar situation.

In [8]:
columns = ['internet_service_type_id', 'internet_service_type']
telco_customers[columns].value_counts()

internet_service_type_id  internet_service_type
2                         Fiber optic              3096
1                         DSL                      2421
3                         None                     1526
dtype: int64

internet_service_type_id is another foreign key columns. We'll keep internet_service_type.

Finally, let's take a look at payment_type_id and payment_type.

In [9]:
columns = ['payment_type_id', 'payment_type']
telco_customers[columns].value_counts()

payment_type_id  payment_type             
1                Electronic check             2365
2                Mailed check                 1612
3                Bank transfer (automatic)    1544
4                Credit card (automatic)      1522
dtype: int64

Another foreign key column. We'll keep payment_type.

customer_id is probably a unique identifier. Let's take a look.

In [10]:
telco_customers['customer_id'].head(3)

0    0002-ORFBO
1    0003-MKNFE
2    0004-TLHLJ
Name: customer_id, dtype: object

In [11]:
telco_customers['customer_id'].nunique() == telco_customers.shape[0]

True

Since the total number of unique values in customer_id matches the total number of rows in the dataframe and the values don't look to be meaningful in any other way we can conclude that this is a unique identifier and will not have any influence over customer churn.

Now we will drop these columns.

In [12]:
columns_to_drop = [
    'customer_id',
    'contract_type_id',
    'internet_service_type_id',
    'payment_type_id'
]

telco_customers = telco_customers.drop(columns = columns_to_drop)

# Let's make sure it worked
telco_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 7043 non-null   object 
 1   senior_citizen         7043 non-null   int64  
 2   partner                7043 non-null   object 
 3   dependents             7043 non-null   object 
 4   tenure                 7043 non-null   int64  
 5   phone_service          7043 non-null   object 
 6   multiple_lines         7043 non-null   object 
 7   online_security        7043 non-null   object 
 8   online_backup          7043 non-null   object 
 9   device_protection      7043 non-null   object 
 10  tech_support           7043 non-null   object 
 11  streaming_tv           7043 non-null   object 
 12  streaming_movies       7043 non-null   object 
 13  paperless_billing      7043 non-null   object 
 14  monthly_charges        7043 non-null   float64
 15  tota

---

## Check Other Features for Unusual Values

Now we will check our remaining columns to determine if there are any unusual values or values that are redundant.

In [13]:
# for each column print a list of unique values
# we will sort the values by index so we can see the head and tail of each column
for column in telco_customers.columns:
    print(telco_customers[column].value_counts().sort_index(), end = '\n----------\n')

Female    3488
Male      3555
Name: gender, dtype: int64
----------
0    5901
1    1142
Name: senior_citizen, dtype: int64
----------
No     3641
Yes    3402
Name: partner, dtype: int64
----------
No     4933
Yes    2110
Name: dependents, dtype: int64
----------
0      11
1     613
2     238
3     200
4     176
     ... 
68    100
69     95
70    119
71    170
72    362
Name: tenure, Length: 73, dtype: int64
----------
No      682
Yes    6361
Name: phone_service, dtype: int64
----------
No                  3390
No phone service     682
Yes                 2971
Name: multiple_lines, dtype: int64
----------
No                     3498
No internet service    1526
Yes                    2019
Name: online_security, dtype: int64
----------
No                     3088
No internet service    1526
Yes                    2429
Name: online_backup, dtype: int64
----------
No                     3095
No internet service    1526
Yes                    2422
Name: device_protection, dtype: int64
-----

- We see a few customers with 0 tenure that will likely not be useful to us since they have not had a chance to churn. 
- There are a lot of columns that have values for no and no service. These can probably be combined into a single no value.
- The total_charges column appears to have some hidden missing values. There are only 11 so we can drop these rows. Dropping these rows is unlikely to affect our results. These may be related to the 0 tenure customers.

Let's remove the 0 tenure customers and see if this takes care of the missing values in total_charges.

In [14]:
# This should remove 11 rows leaving us with 7032 rows

does_not_have_zero_tenure = telco_customers.tenure != 0
telco_customers = telco_customers[does_not_have_zero_tenure]
telco_customers.shape[0]

7032

In [15]:
# Now let's check the total_charges column

telco_customers.total_charges.sort_values().head()

4386     100.2
2222    100.25
1771    100.35
5542     100.4
5126     100.8
Name: total_charges, dtype: object

Now that total_charges consists of only numeric values we can cast the column to a float type.

In [16]:
telco_customers.total_charges = telco_customers.total_charges.astype('float')
telco_customers.total_charges.dtypes

dtype('float64')

That took care of the missing values in total_charges. Next let's handle those redundant values. The columns multiple_lines, online_security, online_backup, device_protection, tech_support, streaming_tv, and streaming_movies all have to "No" values. We will combine these values so that there is only a value for "Yes" and a value for "No".

In [17]:
# We're going to need numpy.where to help with this
import numpy as np

In [18]:
columns = [
    'multiple_lines',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies'
]

for column in columns:
    telco_customers[column] = np.where(telco_customers[column] == 'Yes', 'Yes', 'No')
    
# Let's verify that it worked
for column in columns:
    print(telco_customers[column].value_counts(), end = '\n----------\n')

No     4065
Yes    2967
Name: multiple_lines, dtype: int64
----------
No     5017
Yes    2015
Name: online_security, dtype: int64
----------
No     4607
Yes    2425
Name: online_backup, dtype: int64
----------
No     4614
Yes    2418
Name: device_protection, dtype: int64
----------
No     4992
Yes    2040
Name: tech_support, dtype: int64
----------
No     4329
Yes    2703
Name: streaming_tv, dtype: int64
----------
No     4301
Yes    2731
Name: streaming_movies, dtype: int64
----------


---

## Encode Non Numeric Features

Lastly we need to encode all of our non numeric features into numeric columns so that we can use them with our machine learning models. We will use pandas to help us with this.

In [19]:
# We only need non numeric categorical features
categorical_cols = telco_customers.dtypes[telco_customers.dtypes == 'object'].index

# pandas' get_dummies function will encode these features for us
dummy_df = pd.get_dummies(telco_customers[categorical_cols], dummy_na = False, drop_first = True)
telco_customers = pd.concat([telco_customers, dummy_df], axis = 1)

# Let's see if it worked
telco_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 40 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   gender                                7032 non-null   object 
 1   senior_citizen                        7032 non-null   int64  
 2   partner                               7032 non-null   object 
 3   dependents                            7032 non-null   object 
 4   tenure                                7032 non-null   int64  
 5   phone_service                         7032 non-null   object 
 6   multiple_lines                        7032 non-null   object 
 7   online_security                       7032 non-null   object 
 8   online_backup                         7032 non-null   object 
 9   device_protection                     7032 non-null   object 
 10  tech_support                          7032 non-null   object 
 11  streaming_tv     

Let's just clean up some of those feature names before we conclude.

In [20]:
telco_customers.columns = telco_customers.columns.str.replace(' ', '_', regex = False).str.lower()
telco_customers.columns = telco_customers.columns.str.replace('\(|\)', '', regex = True)
telco_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   gender                              7032 non-null   object 
 1   senior_citizen                      7032 non-null   int64  
 2   partner                             7032 non-null   object 
 3   dependents                          7032 non-null   object 
 4   tenure                              7032 non-null   int64  
 5   phone_service                       7032 non-null   object 
 6   multiple_lines                      7032 non-null   object 
 7   online_security                     7032 non-null   object 
 8   online_backup                       7032 non-null   object 
 9   device_protection                   7032 non-null   object 
 10  tech_support                        7032 non-null   object 
 11  streaming_tv                        7032 no

---

## Create the Function

Now let's put it all in a function.

In [21]:
def prep_telco_data(df):
    df = df.drop_duplicates()

    cols_to_drop = [
        'customer_id',
        'contract_type_id',
        'internet_service_type_id',
        'payment_type_id'
    ]
    df = df.drop(columns = cols_to_drop)

    does_not_have_zero_tenure = df.tenure != 0
    df = df[does_not_have_zero_tenure]
    df.total_charges = df.total_charges.astype('float')

    columns = [
        'multiple_lines',
        'online_security',
        'online_backup',
        'device_protection',
        'tech_support',
        'streaming_tv',
        'streaming_movies'
    ]

    for column in columns:
        df[column] = np.where(df[column] == 'Yes', 'Yes', 'No')

    categorical_cols = df.dtypes[df.dtypes == 'object'].index

    dummy_df = pd.get_dummies(df[categorical_cols], dummy_na = False, drop_first = True)
    df = pd.concat([df, dummy_df], axis = 1)

    df.columns = df.columns.str.replace(' ', '_', regex = False).str.lower()
    df.columns = df.columns.str.replace('\(|\)', '', regex = True)

    return df

Let's test it to make sure it works.

In [22]:
prep_telco_data(get_telco_data()).info()

Using cache
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   gender                              7032 non-null   object 
 1   senior_citizen                      7032 non-null   int64  
 2   partner                             7032 non-null   object 
 3   dependents                          7032 non-null   object 
 4   tenure                              7032 non-null   int64  
 5   phone_service                       7032 non-null   object 
 6   multiple_lines                      7032 non-null   object 
 7   online_security                     7032 non-null   object 
 8   online_backup                       7032 non-null   object 
 9   device_protection                   7032 non-null   object 
 10  tech_support                        7032 non-null   object 
 11  streaming_tv                   

---

## Split the Data

Now that we have our prepare function ready we need to create a function to split the data for us. We need to split the data into train, validate, and test datasets. We will first make an 80/20 split to get a test dataset that is 20% of our original data. Then with the other 80% we will make a 70/30 split where the train dataset will be 70% and the validate dataset will be 30%. We can use sklearn to help with this.

In [23]:
from sklearn.model_selection import train_test_split

# We will use a random seed of 24 for reproducability
seed = 24

# Here we do our 80/20 split
train_validate, test = train_test_split(
    telco_customers,
    test_size = 0.2,
    random_state = seed,
    stratify = telco_customers['churn']
)

# Next we do our 70/30 split
train, validate = train_test_split(
    train_validate,
    test_size = 0.3,
    random_state = seed,
    stratify = train_validate['churn']
)
train.shape, validate.shape, test.shape

((3937, 40), (1688, 40), (1407, 40))

We will put this in a function for our convenience.

In [24]:
def split_data(df, stratify, random_seed = 24):
    test_split = 0.2
    train_validate_split = 0.3

    train_validate, test = train_test_split(
        df,
        test_size = test_split,
        random_state = random_seed,
        stratify = df[stratify]
    )
    
    train, validate = train_test_split(
        train_validate,
        test_size = train_validate_split,
        random_state = random_seed,
        stratify = train_validate[stratify]
    )
    return train, validate, test

Let's test it to make sure it works.

In [25]:
train, validate, test = split_data(telco_customers, 'churn')
train.shape, validate.shape, test.shape

((3937, 40), (1688, 40), (1407, 40))

---

## Conclusion

With the prep_telco_data() and split_data() functions we can now easily prepare and split our data for exploratory analysis and modeling.

**It is important that from this point on we will only work with our train dataset. We will not use validate until we get to modeling and we will not use test until we have selected our best model.**